This notebook displays an example of how to run the mini-grid optimization for multiple settlements at once. It does so using an example settlement file from the Global Electrification Platform (electrifynow.energydata.info) for Sierra Leone.

# Import neccessary packages

In [1]:
from src.mgoptimization.hybrids import *
from src.mgoptimization.pso import *
import os
import numpy as np
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

# Import settlements

Data is retrieved from the example_input folder. Next, 100 sample settlements with more than 1000 people are selected from the file.

In [4]:
df = pd.read_csv('input/sl-2-country-inputs.csv')
df = df.loc[df['PopStartYear'] > 1000]
df = df.sample(100)

# Retrieve solar resource data for the area

Select where to save the PV data

In [5]:
pv_data_folder = 'pv'

Enter your renewables.ninja token

In [6]:
token = 'cdf20d12d762111de60af365755a3e53e6b0b719'

This below cells retrieves the PV data from renewables.ninja using your token.

PV data is not retrieved for each location. Rather, it is retrieved from a set of evenly spaced points within the study area made up of the settlements to be evaluated. Then, the PV data from the closest point is used when optimizing the mini-grids. This approach is used for two reasons:
* To reduce the number of files saved on the computer and associated storage space
* Because Renewables.ninja has allows maximum 50 downloads/hour from the API.

If the API limit is reached, the download will pause for 60 minutes and then proceed. 

In [7]:
resolution = 0.5  # This is the resolution between points, measured in degrees. A finer resolution will increase detail as well as time to retrieve input files.

In [8]:
def roundPartial (value, resolution):
    return round (value / resolution) * resolution

max_lat = roundPartial(df['Y_deg'].max(), resolution)
min_lat = roundPartial(df['Y_deg'].min(), resolution)

max_long = roundPartial(df['X_deg'].max(), resolution)
min_long = roundPartial(df['X_deg'].min(), resolution)

lats = np.arange(min_lat, max_lat + resolution, resolution) #.tolist()
longs = np.arange(min_long, max_long + resolution, resolution) #.tolist()

print('{} data points will be retrieved'.format(len(lats) * len(longs)))

49 data points will be retrieved


In [9]:
for lat in lats:
    for long in longs:
        output_file = os.path.join(pv_data_folder, 'pv_data_lat_{}_long_{}.csv'.format(lat, long))

        if os.path.exists(output_file) == False:
            get_pv_data(lat, long, token, pv_data_folder)

#  Run the optimization to find the best combination of PV, diesel and batteries, as well as the corresponding LCOE

The LCOE of generation, PV capacity (kW), diesel capacity (kW), battery capacity (kWh) and Net Present Cost - NPC (USD) are printed and saved to the Multiple_settlements_example.csv file in the outputs folder.

In [10]:
def run_pso(annual_demand, tier, lat, long):
    hourly_ghi, hourly_temp = read_environmental_data(os.path.join(pv_data_folder, 'pv_data_lat_{}_long_{}.csv'.format(lat, long)))

    load_curve = calc_load_curve(tier, annual_demand)

    result = optimizer(diesel_price=0.5,
                   hourly_ghi=hourly_ghi,
                   hourly_temp=hourly_temp,
                   load_curve=load_curve,
                   start_year=2024,
                   end_year=2034,
                   discount_rate=0.08,
                   diesel_cost=378,  # diesel generator capital cost, USD/kW rated power
                   battery_cost=589,  # battery capital capital cost, USD/kWh of storage capacity
                   full_life_cycles=2000, #full battery cycles before failure
                   battery_inverter_cost=608,  # USD/kW
                   pv_cost=1147,  # PV panel capital cost, USD/kW peak power
                   pv_inverter=0, # PV inverter cost, USD/kW peak power, set to 0 if already included in pv_cost
                   charge_controller=0, # PV charge controller cost, USD/kW peak power, set to 0 if already included in pv_cost
                   diesel_limit=0.5, # Maximum share of generation over a year coming from diesel generator
                   lpsp_max=0.005,  # maximum loss of load allowed over the year
                   iterations=100,
                   n_particles=100,
                   ftol=0.0001,
                   ftol_iter=10,
                   verbose=False)

    lcoe = round(result['gen_lcoe'], 3)
    pv_capacity = round(result['pv_capacity'], 1)
    diesel_capacity = round(result['diesel_gen_capacity'], 1)
    battery_capacity = round(result['battery_capacity'], 1)
    npc = int(round(result['npc'], 0))

    return lcoe, pv_capacity, diesel_capacity, battery_capacity, npc

In [11]:
tier = 3

df['MiniGridLCOE'], df['PVCapacity'], df['DieselCapacity'], df['BatteryCapacity'], df['NPC'] = zip(*df.progress_apply(lambda row: run_pso(
    row['ResidentialDemandTier{}'.format(tier)] * row['PopStartYear'], 
    tier, 
    roundPartial(row['Y_deg'], resolution), 
    roundPartial(row['X_deg'], resolution)), axis=1))

df.to_csv(r'output/Multiple_settlements_example.csv')

df[['id', 'X_deg', 'Y_deg', 'GHI', 'PopStartYear', 'MiniGridLCOE', 'PVCapacity', 'DieselCapacity', 'BatteryCapacity', 'NPC']].sample(10)


  0%|          | 0/100 [00:00<?, ?it/s]

,id,X_deg,Y_deg,GHI,PopStartYear,MiniGridLCOE,PVCapacity,DieselCapacity,BatteryCapacity,NPC
18285,18327,-13.11463,8.93833,1881.40606,4423.580239,0.280,419.9,143.9,190.9,1615727
1852,1853,-11.19665,9.38091,1971.28028,2060.546188,0.265,127.6,72.6,131.6,697277
20370,20415,-10.76858,7.92921,1846.00000,1045.545988,0.248,74.7,31.1,37.6,334142
1618,1619,-12.30263,9.43880,1922.00000,1265.782619,0.266,84.2,34.9,81.6,430007
19636,19681,-12.93489,9.12344,1875.52637,1119.075932,0.260,86.2,31.8,62.2,375967
15861,15892,-11.34040,7.66939,1813.00000,1321.085937,0.251,78.7,37.4,74.0,422772
18761,18803,-10.33601,8.46117,1882.16095,15530.266595,0.245,1114.7,468.0,512.5,4896301
19268,19311,-11.96665,7.65466,1788.94854,1136.140653,0.283,76.2,31.2,122.2,433736
18672,18714,-11.30575,8.57229,1898.00000,1603.712879,0.260,113.1,46.8,94.3,536648
19035,19078,-12.05685,8.11285,1837.00000,4033.636162,0.274,330.3,116.3,238.0,1427336
